<h1 align="center">Eye Diseases Classification 👁️</h1>
<p style="text-align:center;">The dataset consists of Normal, Diabetic Retinopathy, Cataract and Glaucoma retinal images where each class have approximately 1000 images. These images are collected from various sorces like IDRiD, Oculur recognition, HRF etc.</p>

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

from torchmetrics import Accuracy
from torchmetrics import ConfusionMatrix
from torchmetrics.functional import precision_recall
from torchmetrics.functional import f1_score

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial
from ray.air import session

ImportError: cannot import name 'MulticlassAccuracy' from 'torchmetrics.classification' (/Users/sinaraoufi/miniforge3/lib/python3.10/site-packages/torchmetrics/classification/__init__.py)

## Load data

In [ ]:
def load_data():
    t = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256)),
        ])
    
    return datasets.ImageFolder(root='dataset', transform=t)    

## Split data into train-test

In [ ]:
def train_test_split(dataset, train_size, random_state=42):
    train_size = int(train_size * len(dataset))
    test_size = len(dataset) - train_size
    seed = torch.Generator().manual_seed(random_state)
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size], generator=seed)
    
    return train_dataset, test_dataset

In [ ]:
dataset = load_data()

In [ ]:
dataset

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, 0.8)

In [ ]:
# number of classes
K = len(set(dataset.targets))

In [ ]:
print(f'Number of classes: {K}')

## Convolutional neural network (CNN)

In [ ]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.dense_layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(64 * 3 * 3, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, K)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.dense_layers(x)
        
        return x

In [ ]:
model = CNN(K)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

### Exploring images

In [ ]:
def display_image(image, label):
    print(f"Label : {dataset.classes[label]}")
    plt.imshow(image.permute(1,2,0))

# display the first image in the dataset
display_image(*dataset[0])

## DataLoader

In [ ]:
batch_size = 128
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=1)
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=1)

### Visualizing the batch images

In [ ]:
def show_batch(data_loader):
    '''Plot images grid of single batch'''
    for images, labels in data_loader:
        fig, ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(train_dataloader)

### Device configuration

In [ ]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
elif torch.backends.mps.is_available():
    device = "mps"

## Training function

In [ ]:
# A function to encapsulate the training loop
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    
    model.to(device)
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    
    accuracy = MulticlassAccuracy().to(device)
    for epoch in range(epochs):
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device), targets.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        # Get train loss
        train_loss = np.mean(train_loss)
        # Train accuracy
        train_accuracy = accuracy(outputs, targets)
        
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss.append(loss.item())
        
        # Get test loss
        test_loss = np.mean(test_loss) 
        # Test accuracy
        test_accuracy = accuracy(outputs, targets)
        
        # Save losses
        train_losses[epoch] = train_loss
        test_losses[epoch] = test_loss
        
        print(f'Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.2f}, Test Loss: {test_loss:.2f}, \
        Train Accuracy: {train_accuracy:.2f}, Test Accuracy: {test_accuracy:.2f},')
        
    return train_losses, test_losses

In [ ]:
train_losses, test_losses = batch_gd(model, criterion, optimizer, train_dataloader, test_dataloader, epochs=3)

### Plot the losses

In [ ]:
plt.title("Losess")
plt.plot(train_losses, label="Train loss")
plt.plot(test_losses, label="Test loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Predection

In [ ]:
y_pred_list = []
y_true_list = []

with torch.no_grad():
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predections = torch.max(outputs, 1)
        
        y_pred_list.append(targets.cpu().numpy())
        y_true_list.append(predections.cpu().numpy())
        
targets = torch.tensor(np.concatenate(y_true_list))
preds = torch.tensor(np.concatenate(y_pred_list))

## Evaluations

### Confusion matrix

In [ ]:
confmat = ConfusionMatrix(num_classes=K)
cm = confmat(preds, targets)

In [ ]:
sn.heatmap(cm, annot=True, fmt='.0f')
plt.show()

In [ ]:
accuracy = MulticlassAccuracy().to(device)
accuracy = accuracy(preds, targets, num_classes=K)
print(f'Accuracy: {100 * accuracy:.2f}%')

### Precision & Recall

In [ ]:
precision, recall = precision_recall(preds, targets, average='macro', num_classes=K)
print(f'Precision: {100 * precision:.2f}%, Recall: {100 * recall:.2f}%')

### F1-score

In [ ]:
f1_score = f1_score(preds, targets, num_classes=K)
print(f'F1-score: {100 * f1_score:.2f}%')